# Sistema de Recomendação de Produtos (TF-IDF + Similaridade do Cosseno)

Projeto de portfólio — recomendação por **produto** e por **intenção do usuário** usando **NLP clássico**.

**Objetivo:** simular um recomendador de e-commerce (content-based), transformando descrições de produtos em vetores e calculando similaridade para sugerir itens relevantes.


## Sumário
1. Carregamento dos dados  
2. Criação do texto descritivo (feature engineering)  
3. Vetorização TF-IDF  
4. Similaridade do cosseno  
5. Recomendação por nome do produto  
6. Recomendação por descrição (intenção do usuário)  
7. Exibição dos resultados (visual clean)  
8. Exemplos de uso


## 1. Carregamento dos dados


In [1]:
import pandas as pd

In [28]:
df = pd.read_csv("produtos.csv")
df.head()


,product_id,name,category,price_brl,material,brand,tags,rating
0,1,Jogo de Panelas Antiaderente 5pç,Cozinha,349.9,Alumínio,Praticli Home,panelas antiaderente fogão cozinha fácil limpeza,4.6
1,2,Conjunto de Facas Inox 6pç,Cozinha,129.9,Aço Inoxidável,ChefLine,facas inox cozinha corte preciso suporte magné...,4.4
2,3,Assadeira Retangular 3L,Cozinha,69.9,Vidro Temperado,GlassCook,assadeira forno lasanha gratinado vidro,4.5
3,4,Tábua de Corte Bamboo,Cozinha,59.9,Bambu,EcoBoard,tábua corte bamboo sustentável cozinha,4.7
4,5,Airfryer 4L Digital,Cozinha,429.0,PP + Componentes Eletrônicos,AirLite,airfryer fritadeira sem óleo digital saudável,4.8


In [29]:
import pandas as pd, re

def to_float_brl(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    s = re.sub(r"[^\d,.\-]", "", s)
    if "," in s and "." in s:
        s = s.replace(".", "").replace(",", ".")
    elif "," in s and "." not in s:
        s = s.replace(",", ".")
    try:
        return float(s)
    except:
        return None

df["price_brl"] = df["price_brl"].apply(to_float_brl)


## 2. Criação do texto descritivo (feature engineering)
Unifica nome, categoria, material, marca, tags, preço e nota em uma única coluna textual para representar semanticamente cada produto.


In [30]:
def make_text(row):
    parts = [
        str(row["name"]),
        str(row["category"]),
        str(row["material"]),
        str(row["brand"]),
        str(row["tags"]),
        f"preco {row['price_brl']:.2f}",
        f"nota {row['rating']:.1f}"
    ]
    return " ".join(parts)
df["text"] = df.apply(make_text, axis=1)
df[["product_id","name","text"]].head(5)

,product_id,name,text
0,1,Jogo de Panelas Antiaderente 5pç,Jogo de Panelas Antiaderente 5pç Cozinha Alumí...
1,2,Conjunto de Facas Inox 6pç,Conjunto de Facas Inox 6pç Cozinha Aço Inoxidá...
2,3,Assadeira Retangular 3L,Assadeira Retangular 3L Cozinha Vidro Temperad...
3,4,Tábua de Corte Bamboo,Tábua de Corte Bamboo Cozinha Bambu EcoBoard t...
4,5,Airfryer 4L Digital,Airfryer 4L Digital Cozinha PP + Componentes E...


## 3. Vetorização TF-IDF
Transforma o texto em vetores numéricos para permitir comparação.


In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Criamos o transformador TF-IDF
tfidf = TfidfVectorizer()

# Aplicamos ele ao texto dos produtos
X = tfidf.fit_transform(df["text"])

X.shape


(10, 98)

## 4. Similaridade do cosseno
Mede quão semelhantes são os vetores dos produtos entre si.


In [32]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcula a similaridade entre todos os produtos
sim_matrix = cosine_similarity(X)

sim_matrix.shape


(10, 10)

## 5. Recomendação por nome do produto
Busca itens similares a um produto específico (item-based).


In [33]:
import numpy as np

def recomendar(nome_produto, top_n=5):
    # encontra o índice do produto cujo nome foi buscado
    mask = df["name"].str.contains(nome_produto, case=False)
    if not mask.any():
        return "Produto não encontrado. Tente parte do nome (ex: 'Airfryer', 'Tábua')."

    idx = df[mask].index[0]

    # pega as similaridades deste produto para todos os outros
    scores = sim_matrix[idx]

    # ordena do mais parecido para o menos
    indices_similares = np.argsort(scores)[::-1]

    # remove o próprio produto da lista
    indices_similares = [i for i in indices_similares if i != idx]

    # pega os top N mais parecidos
    similares = indices_similares[:top_n]

    return df.loc[similares, ["product_id", "name", "category", "price_brl", "rating"]]


In [34]:
recomendar("Airfryer")

,product_id,name,category,price_brl,rating
3,4,Tábua de Corte Bamboo,Cozinha,59.9,4.7
1,2,Conjunto de Facas Inox 6pç,Cozinha,129.9,4.4
0,1,Jogo de Panelas Antiaderente 5pç,Cozinha,349.9,4.6
2,3,Assadeira Retangular 3L,Cozinha,69.9,4.5
9,10,Potes Herméticos 5pç,Organização,99.9,4.5


## 6. Recomendação por descrição (intenção do usuário)
Converte a frase do cliente para o mesmo espaço vetorial e encontra os itens mais relevantes.


In [35]:
def recomendar_por_descricao(texto_cliente, top_n=5):
    # Transformamos o texto do cliente no mesmo formato dos produtos
    texto_vec = tfidf.transform([texto_cliente])

    # Calculamos a similaridade do texto com todos os produtos
    scores = cosine_similarity(texto_vec, X)[0]

    # Ordenamos os produtos do mais parecido para o menos
    indices_similares = np.argsort(scores)[::-1]

    # Pegamos os top N produtos mais relevantes
    similares = indices_similares[:top_n]

    return df.loc[similares, ["product_id", "name", "category", "price_brl", "rating"]]


In [37]:
recomendar_por_descricao("quero itens sustentáveis")


,product_id,name,category,price_brl,rating
9,10,Potes Herméticos 5pç,Organização,99.9,4.5
8,9,Kit Organizadores de Geladeira 6pç,Organização,119.9,4.4
7,8,Jogo Americano 4pç Linho,Mesa,79.9,4.2
6,7,Garrafa Térmica 1L Inox,Mesa,139.9,4.6
5,6,Jogo de Copos 6x 300ml,Mesa,49.9,4.3


In [38]:
recomendar_por_descricao("quero organizar minha despensa")



,product_id,name,category,price_brl,rating
9,10,Potes Herméticos 5pç,Organização,99.9,4.5
8,9,Kit Organizadores de Geladeira 6pç,Organização,119.9,4.4
7,8,Jogo Americano 4pç Linho,Mesa,79.9,4.2
6,7,Garrafa Térmica 1L Inox,Mesa,139.9,4.6
5,6,Jogo de Copos 6x 300ml,Mesa,49.9,4.3


In [39]:
recomendar_por_descricao("quero algo prático para cozinha")


,product_id,name,category,price_brl,rating
3,4,Tábua de Corte Bamboo,Cozinha,59.9,4.7
1,2,Conjunto de Facas Inox 6pç,Cozinha,129.9,4.4
0,1,Jogo de Panelas Antiaderente 5pç,Cozinha,349.9,4.6
2,3,Assadeira Retangular 3L,Cozinha,69.9,4.5
9,10,Potes Herméticos 5pç,Organização,99.9,4.5


In [53]:
# tenta converter a coluna para float; se algo não converter, vira NaN
df["price_brl"] = pd.to_numeric(df["price_brl"], errors="coerce")


## 7. Exibição dos resultados (visual clean)
Formatação apenas para visualização (não altera os dados).


In [50]:
def mostrar_recomendacao_clean(df_result, titulo="Recomendações"):
    df_show = df_result.copy()

    def brl(v):
        if pd.isna(v): return ""
        return f"R${v:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")

    if "price_brl" in df_show.columns:
        df_show["price_brl"] = df_show["price_brl"].apply(brl)
    if "rating" in df_show.columns:
        df_show["rating"] = df_show["rating"].map(lambda x: f"{x:.1f}")

    df_show = df_show.rename(columns={
        "product_id": "ID",
        "name": "Produto",
        "category": "Categoria",
        "price_brl": "Preço",
        "rating": "Avaliação"
    })

    styled = (
        df_show.style
        .set_caption(titulo)
        .hide(axis="index")
        .set_table_styles([
            {"selector": "caption", "props": "caption-side: top; text-align: left; font-weight: 600; font-size: 15px; padding: 4px 0;"},
            {"selector": "th", "props": "background-color: #f4f4f4; color: #111; font-weight: 600; border-bottom: 1px solid #ddd;"},
            {"selector": "td", "props": "padding: 6px 8px; border-bottom: 1px solid #eee;"},
            {"selector": "table", "props": "border-collapse: collapse; margin: 8px 0;"},
        ])
        .set_properties(**{"font-size": "14px", "color": "#111"})
        .set_properties(subset=["ID","Preço","Avaliação"], **{"text-align": "right"})
        .set_properties(subset=["Produto","Categoria"], **{"text-align": "left"})
    )
    display(styled)


In [51]:
def brl(v):
    if pd.isna(v):
        return ""
    return f"R${v:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")


## 8. Exemplos de uso


In [54]:
res1 = recomendar("Airfryer")
mostrar_recomendacao_clean(res1, "Recomendações — similares a: Airfryer")




ID,Produto,Categoria,Preço,Avaliação
4,Tábua de Corte Bamboo,Cozinha,"R$59,90",4.7
2,Conjunto de Facas Inox 6pç,Cozinha,"R$129,90",4.4
1,Jogo de Panelas Antiaderente 5pç,Cozinha,"R$349,90",4.6
3,Assadeira Retangular 3L,Cozinha,"R$69,90",4.5
10,Potes Herméticos 5pç,Organização,"R$99,90",4.5


In [55]:
res2 = recomendar_por_descricao("quero presente útil para alguém que gosta de cozinhar")
mostrar_recomendacao_clean(res2, "Recomendações — intenção: presente útil para quem cozinha")



ID,Produto,Categoria,Preço,Avaliação
4,Tábua de Corte Bamboo,Cozinha,"R$59,90",4.7
6,Jogo de Copos 6x 300ml,Mesa,"R$49,90",4.3
2,Conjunto de Facas Inox 6pç,Cozinha,"R$129,90",4.4
1,Jogo de Panelas Antiaderente 5pç,Cozinha,"R$349,90",4.6
9,Kit Organizadores de Geladeira 6pç,Organização,"R$119,90",4.4


## 9. Conclusão

Neste projeto construí um sistema de recomendação de produtos baseado em conteúdo, utilizando TF-IDF para representar textos e similaridade do cosseno para encontrar itens mais próximos.  
Foram criados dois modos de recomendação: por nome do produto e por intenção do usuário em linguagem natural, simulando um cenário de e-commerce real.  

O modelo apresentou resultados coerentes e consistentes, capturando contexto sem necessidade de deep learning.  
Como próximos passos, é possível substituir TF-IDF por embeddings (ex.: Word2Vec ou BERT) para melhorar a compreensão semântica e expandir o sistema para uso em uma API ou interface web.
